In [ ]:
import psfr
from psfr.psfr import stack_psf, one_step_psf_estimate, _linear_amplitude, centroid_fit, shift_psf
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Util import util
from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Testing
This is incomplete and needs to be improved during packaging

In [ ]:
def test_one_step_psf_estimation():
    
    from lenstronomy.LightModel.light_model import LightModel
    numpix = 21
    n_c = (numpix - 1) / 2
    x_grid, y_grid = util.make_grid(numPix=21, deltapix=1, left_lower=True)
    gauss = LightModel(['GAUSSIAN'])
    x_c, y_c = -0.6, 0.2
    sigma = 1
    kwargs_true = [{'amp': 1, 'sigma': sigma, 'center_x': n_c, 'center_y': n_c}]
    flux_true = gauss.surface_brightness(x_grid, y_grid, kwargs_true)
    psf_true = util.array2image(flux_true)
    psf_true /= np.sum(psf_true)
    
    kwargs_guess = [{'amp': 1, 'sigma': 1.5, 'center_x': n_c, 'center_y': n_c}]
    flux_guess = gauss.surface_brightness(x_grid, y_grid, kwargs_guess)
    psf_guess = util.array2image(flux_guess)
    psf_guess /= np.sum(psf_guess)
    
    center_list = []
    star_list = []
    displacement_scale = 1
    
    x_c, y_c = 0., 4.5
    center_list.append(np.array([x_c, y_c]))
    kwargs_model = [{'amp': 1, 'sigma': sigma, 'center_x': n_c + x_c, 'center_y': n_c + y_c}]
    flux_model = gauss.surface_brightness(x_grid, y_grid, kwargs_model)
    star = util.array2image(flux_model)
    star_list.append(star)
    
    
    
    oversampling = 2
    numpix_super = numpix * oversampling
    if oversampling % 2 == 0:
        numpix_super -= 1
    
    x_grid_super, y_grid_super = util.make_grid(numPix=numpix_super, deltapix=1. / oversampling, left_lower=True)
    flux_guess_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_guess)
    psf_guess_super = util.array2image(flux_guess_super)
    psf_guess_super /= np.sum(psf_guess_super)
    
    flux_true_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_true)
    psf_true_super = util.array2image(flux_true_super)
    psf_true_super /= np.sum(psf_true_super)

    psf_after_super = one_step_psf_estimate(star_list, psf_guess_super, center_list, mask_list=None, 
                                      error_map_list=None, step_factor=0.2, oversampling=oversampling, verbose=True)
    
    # psf_after = one_step_psf_estimate(star_list, psf_guess, center_list, mask_list=None, error_map_list=None, step_factor=0.2, verbose=True)
    # psf_after should be a better guess of psf_true than psf_guess
    diff_after = np.sum((psf_after_super - psf_true_super)**2)
    diff_before = np.sum((psf_guess_super - psf_true_super)**2)
    
    plt.imshow(psf_true_super - psf_after_super)
    plt.colorbar()
    plt.title('after')
    plt.show()
    plt.imshow(psf_true_super - psf_guess_super)
    plt.colorbar()
    plt.title('before')
    plt.show()
    
    plt.imshow(psf_after_super - psf_guess_super)
    plt.colorbar()
    plt.title('after - before')
    plt.show()
    
    assert diff_after < diff_before
    
    
    
test_one_step_psf_estimation()

In [ ]:
import numpy.testing as npt

def test_linear_amplitude():
    amp = 2
    data = np.ones((5, 5)) * amp
    model = np.ones((5, 5))

    amp_return = _linear_amplitude(data, model)
    npt.assert_almost_equal(amp_return, amp)
    
    mask = np.ones_like(data)
    
    amp_return = _linear_amplitude(data, model, mask=mask)
    npt.assert_almost_equal(amp_return, amp)
    
test_linear_amplitude()


def test_fit_centroid():


    from lenstronomy.LightModel.light_model import LightModel
    numpix = 21
    n_c = (numpix - 1) / 2
    x_grid, y_grid = util.make_grid(numPix=21, deltapix=1, left_lower=True)
    gauss = LightModel(['GAUSSIAN'])
    x_c, y_c = -0.6, 0.2
    kwargs_true = [{'amp': 2, 'sigma': 1, 'center_x': n_c + x_c, 'center_y': n_c + y_c}]
    kwargs_model = [{'amp': 1, 'sigma': 1, 'center_x': n_c, 'center_y': n_c}]
    flux_true = gauss.surface_brightness(x_grid, y_grid, kwargs_true)
    flux_true = util.array2image(flux_true)

    flux_model = gauss.surface_brightness(x_grid, y_grid, kwargs_model)
    flux_model = util.array2image(flux_model)

    mask = np.ones_like(flux_true)

    center = centroid_fit(flux_true, flux_model, mask=mask, variance=None)
    npt.assert_almost_equal(center[0], x_c, decimal=1)
    npt.assert_almost_equal(center[1], y_c, decimal=1)
    
test_fit_centroid()


def test_one_step_psf_estimation():
    
    from lenstronomy.LightModel.light_model import LightModel
    numpix = 21
    x_grid, y_grid = util.make_grid(numPix=21, deltapix=1, left_lower=False)
    gauss = LightModel(['GAUSSIAN'])
    x_c, y_c = -0.6, 0.2
    sigma = 1
    kwargs_true = [{'amp': 1, 'sigma': sigma, 'center_x': 0, 'center_y': 0}]
    flux_true = gauss.surface_brightness(x_grid, y_grid, kwargs_true)
    psf_true = util.array2image(flux_true)
    psf_true /= np.sum(psf_true)
    
    oversampling = 4
    
    x_grid_super, y_grid_super = util.make_grid(numPix=21 * oversampling + 1, deltapix=1./oversampling, left_lower=False)
    flux_true_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_true)
    psf_true_super = util.array2image(flux_true_super)
    psf_true_super /= np.sum(psf_true_super)
    
    
    kwargs_guess = [{'amp': 1, 'sigma': 1.2, 'center_x': 0, 'center_y': 0}]
    flux_guess = gauss.surface_brightness(x_grid, y_grid, kwargs_guess)
    psf_guess = util.array2image(flux_guess)
    psf_guess /= np.sum(psf_guess)
    
    center_list = []
    star_list = []
    displacement_scale = 1
    for i in range(8):
        x_c, y_c = np.random.uniform(-0.5, 0.5) * displacement_scale, np.random.uniform(-0.5, 0.5) * displacement_scale
        center_list.append(np.array([x_c, y_c]))
        shift=np.array([x_c, y_c])
        star = shift_psf(psf_center=psf_true_super, oversampling=oversampling, shift=shift, degrade=True, n_pix_star=len(psf_true))
        #kwargs_model = [{'amp': 1, 'sigma': sigma, 'center_x': n_c + x_c, 'center_y': n_c + y_c}]
        #flux_model = gauss.surface_brightness(x_grid, y_grid, kwargs_model)
        #star = util.array2image(flux_model)
        star_list.append(star)
    
    
    psf_after = one_step_psf_estimate(star_list, psf_guess, center_list, mask_list=None, error_map_list=None, step_factor=0.2)
    # psf_after should be a better guess of psf_true than psf_guess
    diff_after = np.sum((psf_after - psf_true)**2)
    diff_before = np.sum((psf_guess - psf_true)**2)
    
    plt.imshow(psf_true - psf_after)
    plt.colorbar()
    plt.title('after')
    plt.show()
    plt.imshow(psf_true - psf_guess)
    plt.colorbar()
    plt.title('before')
    plt.show()
    
    plt.imshow(psf_after - psf_guess)
    plt.colorbar()
    plt.title('after - before')
    plt.show()
    
    print(np.sum(psf_after), np.sum(psf_true))
    
    assert diff_after < diff_before
    

    
    oversampling = 2
    numpix_super = numpix * oversampling
    if oversampling % 2 == 0:
        numpix_super -= 1
    
    x_grid_super, y_grid_super = util.make_grid(numPix=numpix_super, deltapix=1. / oversampling, left_lower=True)
    flux_guess_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_guess)
    psf_guess_super = util.array2image(flux_guess_super)
    psf_guess_super /= np.sum(psf_guess_super)
    
    flux_true_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_true)
    psf_true_super = util.array2image(flux_true_super)
    psf_true_super /= np.sum(psf_true_super)

    psf_after_super = one_step_psf_estimate(star_list, psf_guess_super, center_list, mask_list=None, 
                                      error_map_list=None, step_factor=0.2, oversampling=oversampling)
    diff_after = np.sum((psf_after_super - psf_true_super)**2)
    diff_before = np.sum((psf_guess_super - psf_true_super)**2)
    assert diff_after < diff_before
    
    
test_one_step_psf_estimation()


In [ ]:
def test_stack_psf():
    
    from lenstronomy.LightModel.light_model import LightModel
    numpix = 21
    n_c = (numpix - 1) / 2
    oversampling = 4
    x_grid, y_grid = util.make_grid(numPix=21, deltapix=1, left_lower=True)
    x_grid_super, y_grid_super = util.make_grid(numPix=21, deltapix=1, left_lower=True)
    gauss = LightModel(['GAUSSIAN'])
    sigma = 1
    kwargs_true = [{'amp': 1, 'sigma': sigma, 'center_x': n_c, 'center_y': n_c}]
    flux_true = gauss.surface_brightness(x_grid, y_grid, kwargs_true)
    psf_true = util.array2image(flux_true)
    psf_true /= np.sum(psf_true)
    
    kwargs_guess = [{'amp': 1, 'sigma': 1.5, 'center_x': n_c, 'center_y': n_c}]
    flux_guess = gauss.surface_brightness(x_grid, y_grid, kwargs_guess)
    psf_guess = util.array2image(flux_guess)
    psf_guess /= np.sum(psf_guess)
    
    center_list = []
    star_list = []
    scatter_scale = 1
    for i in range(10):
        x_c, y_c = np.random.uniform(-0.5, 0.5)*scatter_scale, np.random.uniform(-0.5, 0.5)*scatter_scale
        center_list.append(np.array([x_c, y_c]))
        amp = np.random.uniform([0.1, 10])
        kwargs_model = [{'amp': 1, 'sigma': sigma, 'center_x': n_c + x_c, 'center_y': n_c + y_c}]
        flux_model = gauss.surface_brightness(x_grid, y_grid, kwargs_model)
        star = util.array2image(flux_model)
        star_list.append(star)
    

    psf_after, center_list_after, mask_list = stack_psf(star_list, oversampling=1, saturation_limit=None, num_iteration=100, n_recenter=4)
    
    plt.imshow(psf_true - psf_after)
    plt.title('psf_true - psf_after')
    plt.colorbar()
    plt.show()
    plt.imshow(psf_true - psf_guess)
    plt.title('psf_true - psf_guess')
    plt.colorbar()
    plt.show()
    assert np.sum((psf_after - psf_true)**2) < np.sum((psf_guess - psf_true)**2)
    assert len(psf_after) == len(star_list[0])
    
    
    oversampling = 4
    numpix_super = numpix * oversampling
    if oversampling % 2 == 0:
        numpix_super -= 1
    
    x_grid_super, y_grid_super = util.make_grid(numPix=numpix_super, deltapix=1. / oversampling, left_lower=True)
    flux_guess_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_guess)
    psf_guess_super = util.array2image(flux_guess_super)
    psf_guess_super /= np.sum(psf_guess_super)
    
    flux_true_super = gauss.surface_brightness(x_grid_super, y_grid_super, kwargs_true)
    psf_true_super = util.array2image(flux_true_super)
    psf_true_super /= np.sum(psf_true_super)

    psf_after_super, center_list_after, mask_list = stack_psf(star_list, oversampling=oversampling, 
                                                              saturation_limit=None, num_iteration=100, 
                                                              n_recenter=200, verbose=False, kwargs_one_step={'verbose': False})

    diff_after = np.sum((psf_after_super - psf_true_super)**2)
    diff_before = np.sum((psf_guess_super - psf_true_super)**2)
    
    
    plt.imshow(psf_true_super - psf_after_super)
    plt.title('psf_true - psf_after')
    plt.colorbar()
    plt.show()
    plt.imshow(psf_true_super - psf_guess_super)
    plt.title('psf_true - psf_before')
    plt.colorbar()
    plt.show()
    plt.imshow(psf_after_super)
    plt.colorbar()
    plt.title('psf_after')
    plt.show()
    assert diff_after < diff_before
    
test_stack_psf()